The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [2]:
%%capture

import sys
sys.path.append('..')
import mock_d2l_jax as d2l

The function below takes three arguments corresponding to the number
of convolutional layers `num_convs`, the number of input channels `in_channels`
and the number of output channels `out_channels`.


In [3]:
import jax
from jax import numpy as jnp, random, grad, vmap, jit
from flax import linen as nn
import optax
# from d2l import jax as w


def vgg_block(num_convs, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv(out_channels, kernel_size=(3, 3), padding=(1, 1)))
        layers.append(nn.relu)
    layers.append(lambda x: nn.max_pool(x, window_shape=(2, 2), strides=(2, 2)))
    return nn.Sequential(layers)

## [**VGG Network**]
:label:`subsec_vgg-network`

Like AlexNet and LeNet, 
the VGG Network can be partitioned into two parts:
the first consisting mostly of convolutional and pooling layers
and the second consisting of fully connected layers that are identical to those in AlexNet. 
The key difference is 
that the convolutional layers are grouped in nonlinear transformations that 
leave the dimensonality unchanged, followed by a resolution-reduction step, as 
depicted in :numref:`fig_vgg`. 

![From AlexNet to VGG that is designed from building blocks.](http://d2l.ai/_images/vgg.svg)
:width:`400px`
:label:`fig_vgg`

The convolutional part of the network connects several VGG blocks from :numref:`fig_vgg` (also defined in the `vgg_block` function)
in succession. This grouping of convolutions is a pattern that has 
remained almost unchanged over the past decade, although the specific choice of 
operations has undergone considerable modifications. 
The variable `conv_arch` consists of a list of tuples (one per block),
where each contains two values: the number of convolutional layers
and the number of output channels,
which are precisely the arguments required to call
the `vgg_block` function. As such, VGG defines a *family* of networks rather than just 
a specific manifestation. To build a specific network we simply iterate over `arch` to compose the blocks.


In [4]:
class VGG(d2l.Classifier):
    arch: list
    lr: float = 0.1
    num_classes: int = 10
    
    def setup(self):
        conv_blks = []
        for (num_convs, out_channels) in self.arch:
            conv_blks.append(vgg_block(num_convs, out_channels))
        self.net = nn.Sequential([
            *conv_blks, d2l.flatten,
            nn.Dense(4096), nn.relu, nn.Dropout(0.5, deterministic=False),
            nn.Dense(4096), nn.relu, nn.Dropout(0.5, deterministic=False),
            nn.Dense(self.num_classes)])
        # self.net.apply(d2l.init_cnn)

The original VGG network had 5 convolutional blocks,
among which the first two have one convolutional layer each
and the latter three contain two convolutional layers each.
The first block has 64 output channels
and each subsequent block doubles the number of output channels,
until that number reaches 512.
Since this network uses 8 convolutional layers
and 3 fully connected layers, it is often called VGG-11.


In [5]:
rngs = {'params': random.PRNGKey(0), 'dropout': random.PRNGKey(1)}

VGG(arch=((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))).layer_summary(
    (1, 224, 224, 1), rngs=rngs)

                                   VGG Summary                                   
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path                  ┃ outputs                ┃ params                       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Inputs                │ float32[1,224,224,1]   │                              │
├───────────────────────┼────────────────────────┼──────────────────────────────┤
│ net/layers_0/layers_0 │ float32[1,224,224,64]  │ bias: float32[64]            │
│                       │                        │ kernel: float32[3,3,1,64]    │
│                       │                        │                              │
│                       │                        │ 640 (2.6 KB)                 │
├───────────────────────┼────────────────────────┼──────────────────────────────┤
│ net/layers_0          │ float32[1,112,112,64]  │                              │
├───────────────────────┼────────────────────────┼──────────────────────────────┤
│ net/layers_1/layers_0 │ float32[1,112,112,128] │ bias: float32[128]           │
│                       │                        │ kernel: float32[3,3,64,128]  │
│                       │                        │                              │
│                       │                        │ 73,856 (295.4 KB)            │
├───────────────────────┼────────────────────────┼──────────────────────────────┤
│ net/layers_1          │ float32[1,56,56,128]   │                              │
├───────────────────────┼────────────────────────┼──────────────────────────────┤
│ net/layers_11         │ float32[1,4096]        │                              │
├───────────────────────┼────────────────────────┼──────────────────────────────┤
│ net/layers_12         │ float32[1,10]          │ bias: float32[10]            │
│                       │                        │ kernel: float32[4096,10]     │
│                       │                        │                              │
│                       │                        │ 40,970 (163.9 KB)            │
├───────────────────────┼────────────────────────┼──────────────────────────────┤
│ net/layers_2/layers_0 │ float32[1,56,56,256]   │ bias: float32[256]           │
│                       │                        │ kernel: float32[3,3,128,256] │
│                       │                        │                              │
│                       │                        │ 295,168 (1.2 MB)             │
├───────────────────────┼────────────────────────┼──────────────────────────────┤
│ net/layers_2/layers_2 │ float32[1,56,56,256]   │ bias: float32[256]           │
│                       │                        │ kernel: float32[3,3,256,256] │
│                       │                        │                              │
│                       │                        │ 590,080 (2.4 MB)             │
├───────────────────────┼────────────────────────┼──────────────────────────────┤
│ net/layers_2          │ float32[1,28,28,256]   │                              │
├───────────────────────┼────────────────────────┼──────────────────────────────┤
│ net/layers_3/layers_0 │ float32[1,28,28,512]   │ bias: float32[512]           │
│                       │                        │ kernel: float32[3,3,256,512] │
│                       │                        │                              │
│                       │                        │ 1,180,160 (4.7 MB)           │
├───────────────────────┼────────────────────────┼──────────────────────────────┤
│ net/layers_3/layers_2 │ float32[1,28,28,512]   │ bias: float32[512]           │
│                       │                        │ kernel: float32[3,3,512,512] │
│                       │                        │                              │
│                       │                        │ 2,359,808 (9.4 MB)           │
├───────────────────────┼────────────────────────┼──────────────

As you can see, we halve height and width at each block,
finally reaching a height and width of 7
before flattening the representations
for processing by the fully connected part of the network.

## Training

[**Since VGG-11 is more computationally-heavy than AlexNet
we construct a network with a smaller number of channels.**]
This is more than sufficient for training on Fashion-MNIST.
The [**model training**] process is similar to that of AlexNet in :numref:`sec_alexnet`.


In [ ]:
model = VGG(arch=((1, 16), (1, 32), (2, 64), (2, 128), (2, 128)), lr=0.01)
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
data = d2l.FashionMNIST(batch_size=128, resize=(224, 224))
# model.apply_init([next(iter(data.get_dataloader(True)))[0]], d2l.init_cnn)
trainer.fit(model, data, rngs=rngs)

## Summary

One might argue that VGG is the first truly modern convolutional neural network. While AlexNet introduced many of the components of what make deep learning effective at scale, it is VGG that arguably introduced key properties such as blocks of multiple convolutions and a preference for deep and narrow networks. It is also the first network that is actually an entire family of similarly parametrized models, giving the practitioner ample trade-off between complexity and speed. This is also the place where modern deep learning frameworks shine. It is no longer necessary to generate XML config files to specify a network but rather, to assmple said networks through simple Python code. 

Very recently ParNet :cite:`Goyal.Bochkovskiy.Deng.ea.2021` demonstrated that it is possible to achieve competitive performance using a much more shallow architecture through a large number of parallel computations. This is an exciting development and there's hope that it will influence architecture designs in the future. For the remainder of the chapter, though, we will follow the path of scientific progress over the past decade. 

## Exercises


1. Compared with AlexNet, VGG is much slower in terms of computation, and it also needs more GPU memory. 
    1. Compare the number of parameters needed for AlexNet and VGG.
    1. Compare the number of floating point operations used in the convolutional layers and in the fully connected layers. 
    1. How could you reduce the computational cost created by the fully connected layers?
1. When displaying the dimensions associated with the various layers of the network, we only see the information 
   associated with 8 blocks (plus some auxiliary transforms), even though the network has 11 layers. Where did 
   the remaining 3 layers go?
1. Upsampling the resolution in Fashion-MNIST by a factor of $8 \times 8$ from 28 to 224 dimensions is highly 
   wasteful. Try modifying the network architecture and resolution conversion, e.g., to 56 or to 84 dimensions 
   for its input instead. Can you do so without reducing the accuracy of the network?
1. Use Table 1 in the VGG paper :cite:`Simonyan.Zisserman.2014` to construct other common models, 
   such as VGG-16 or VGG-19.


[Discussions](https://discuss.d2l.ai/t/78)
